Example notebook showing how to use the Ensemble sampler (currently in development)

In [1]:
import os
import sys
import argparse
import torch
from getdist import plots, MCSamples
import getdist
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import emcee
import corner

In [2]:
path = os.path.realpath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, path)

In [3]:
from nnest import MCMCSampler
from nnest.likelihoods import *
from nnest.priors import *

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [5]:
# Likelihood
#like = Himmelblau(2)
#prior = UniformPrior(2, -5, 5)
#like = Rosenbrock(2)
#prior = UniformPrior(2, -2, 10)
#like = Gaussian(2, 0.9)
#prior = UniformPrior(2, -5, 5)
#like = Eggbox(2)
#prior = UniformPrior(2, -15, 15)
#like = GaussianShell(2)
#prior = UniformPrior(2, -3, 3)
#like = GaussianMix(2)
#prior = UniformPrior(2, -8, 8)
like = DoubleGaussianShell(2, centers=[[-4.0, 0.0], [4.0, 0.0]], weights=[0.5, 1.0])
prior = UniformPrior(2, [-7, -3], [7, 3])

In [6]:
sampler = MCMCSampler(like.x_dim, like, flow='spline', prior=prior)

Creating directory for new run logs/test/run12
[nnest.trainer] [INFO] SingleSpeedSpline(
  (flow): NormalizingFlow(
    (flows): ModuleList(
      (0): ActNorm()
      (1): Invertible1x1Conv()
      (2): NSF_CL(
        (f1): MLP(
          (net): Sequential(
            (0): Linear(in_features=1, out_features=16, bias=True)
            (1): LeakyReLU(negative_slope=0.2)
            (2): Linear(in_features=16, out_features=16, bias=True)
            (3): LeakyReLU(negative_slope=0.2)
            (4): Linear(in_features=16, out_features=16, bias=True)
            (5): LeakyReLU(negative_slope=0.2)
            (6): Linear(in_features=16, out_features=23, bias=True)
          )
        )
        (f2): MLP(
          (net): Sequential(
            (0): Linear(in_features=1, out_features=16, bias=True)
            (1): LeakyReLU(negative_slope=0.2)
            (2): Linear(in_features=16, out_features=16, bias=True)
            (3): LeakyReLU(negative_slope=0.2)
            (4): Linear(in_fe

In [ ]:
sampler.run(2000, 5, bootstrap_iters=1, bootstrap_num_walkers=200)

[nnest.sampler] [INFO] Initial acceptance [0.2470]
[nnest.trainer] [INFO] Number of training samples [200]
[nnest.trainer] [INFO] Training jitter [0.0100]
[nnest.trainer] [INFO] Epoch [1] train loss [0.0333] validation loss [0.1509]
[nnest.trainer] [INFO] Epoch [100] train loss [0.0100] validation loss [0.0791]
[nnest.trainer] [INFO] Epoch [134] ran out of patience
[nnest.trainer] [INFO] Best epoch [84] validation loss [0.0720] train time (s) [7.6039]]


/Users/adammoss/opt/anaconda3/envs/nnest/lib/python3.8/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in double_scalars
  lnpdiff = f + nlp - state.log_prob[j]


[nnest.sampler] [INFO] Acceptance [0.4281] min ESS [0.3388] max ESS [1.0527] average jump [0.3863]
Removed no burn in
[nnest.trainer] [INFO] Number of training samples [415]
[nnest.trainer] [INFO] Training jitter [0.0100]
[nnest.trainer] [INFO] Epoch [1] train loss [0.0313] validation loss [0.0516]
[nnest.trainer] [INFO] Epoch [100] train loss [0.0074] validation loss [0.0130]
[nnest.trainer] [INFO] Epoch [200] train loss [0.0069] validation loss [0.0194]
[nnest.trainer] [INFO] Epoch [219] ran out of patience
[nnest.trainer] [INFO] Best epoch [169] validation loss [0.0082] train time (s) [22.4287]]
[nnest.sampler] [INFO] Step [100] acceptance [0.2620] min ESS [3.0592] max ESS [11.3017] average jump [1.0501]
[nnest.sampler] [INFO] Step [200] acceptance [0.2570] min ESS [5.6862] max ESS [17.3429] average jump [1.0287]
[nnest.sampler] [INFO] Step [300] acceptance [0.2580] min ESS [9.4213] max ESS [25.2007] average jump [1.0042]
[nnest.sampler] [INFO] Step [400] acceptance [0.2560] min ESS

In [ ]:
like.num_evaluations

In [ ]:
fig = plt.figure(figsize=(8,6))
for i in range(1):
    plt.plot(sampler.samples[i,:,0], sampler.samples[i,:,1])
plt.show()

In [ ]:
fig, ax = plt.subplots(like.x_dim, 1, figsize=(10, like.x_dim), sharex=True)
for i in range(like.x_dim):
    ax[i].plot(sampler.samples[0,:,i])
plt.show()

In [ ]:
flat_samples = sampler.samples[:,50:,:]
flat_samples = flat_samples.reshape((-1, flat_samples.shape[2]))

In [ ]:
fig = corner.corner(flat_samples)

In [ ]:
mc = MCSamples(samples=[sampler.samples[i, :, :].squeeze() for i in range(sampler.samples.shape[0])], 
               loglikes=[-sampler.loglikes[i, :].squeeze() for i in range(sampler.loglikes.shape[0])])

In [ ]:
print(mc.getEffectiveSamples())
print(mc.getMargeStats())
print(mc.getConvergeTests())

In [ ]:
g = plots.getSubplotPlotter(width_inch=8)
g.triangle_plot(mc, filled=True)